In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.chdir('..')

import pickle

import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

from tqdm import tqdm

from model import MemoryCell

In [2]:
from train import run_single_experiment

In [3]:
import pandas as pd
# df = pd.DataFrame({'text': texts})
# df.to_csv('pg19_valid_1k_chunks.csv')
df = pd.read_csv('./notebooks/pg19_valid_1k_chunks.csv', index_col=0)
df.head()

,text
0,"'Let us stay here,' she exclaimed. 'The one ro..."
1,These lobsters don't realize that Jonesy's fas...
2,It certainly was not relief that he felt on di...
3,Down to 1765 the duty imposed was only one pen...
4,"And what was, perhaps, more extraordinary, tho..."


In [4]:
model_name = 'princeton-nlp/Sheared-LLaMA-1.3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_length = 128
N_mem_tokens = 1
device = 'cuda'
dtype = torch.bfloat16
use_flash_attention_2 = True

lr = 1e-02
beta_1 = 0.9
beta_2 = 0.9
weight_decay = 0.01
num_iterations = 5000

samples = df['text'][:50]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [26]:
from tqdm.auto import tqdm
sample_idx = 4
for sample_idx in range(5, 10):
    results = []
    for run_idx in tqdm(range(50), desc=f'sample_idx={sample_idx}'):
        result = run_single_experiment(N_mem_tokens, samples[sample_idx], max_length, num_iterations, sample_idx, run_idx+1, 
                                    model_name, dtype, use_flash_attention_2, device, tokenizer,
                                    lr, beta_1, beta_2, weight_decay)
        results += [result]
    pickle.dump(results, open(f'./runs/{model_name}/mem_{N_mem_tokens}_len_{max_length}_sample_{sample_idx}.pkl', 'wb'))

sample_idx=5:   0%|          | 0/50 [00:00<?, ?it/s]

sample_idx=6:   0%|          | 0/50 [00:00<?, ?it/s]

sample_idx=7:   0%|          | 0/50 [00:00<?, ?it/s]

sample_idx=8:   0%|          | 0/50 [00:00<?, ?it/s]

sample_idx=9:   0%|          | 0/50 [00:00<?, ?it/s]

In [20]:
result.keys()

dict_keys(['losses', 'accuracies', 'original_loss', 'original_accuracy', 'best_memory_params', 'best_loss', 'best_accuracy', 'max_length', 'n_mem_tokens', 'args'])